# Preprocessing OASIS1

In [1]:
# IMPORTS
# DATA MANIPULATION
import pandas as pd
import numpy as np

# DATA VISUALISATION
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn import set_config; set_config(display='diagram') # Visualize pipelines in HTML
import graphviz
from sklearn.tree import export_graphviz
from sklearn import tree

# MACHINE LEARNING
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import xgboost as xgb

/home/cynthias13w/.pyenv/versions/3.8.12/envs/memobrain/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Importing Data
oasis1_cs_path = '../raw_data/OASIS1/oasis_cross-sectional.csv'
oasis1 = pd.read_csv(oasis1_cs_path)

In [3]:
# Checking shape
oasis1.shape

(436, 12)

📜 <big> Note: </big>


* **`CDR`**  Clinical Dementia Rating 
    * 0= nondemented;
    * 0.5 – very mild dementia;
    * 1 = mild dementia;
    * 2 = moderate dementia

* **`eTIV`** Estimated total intracranial volume (eTIV)

* **`nWBV`** Normalized whole brain volume

* **`ASF`** Atlas scaling factor

In [4]:
oasis1.sample(20)

,ID,M/F,Hand,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF,Delay
37,OAS1_0041_MR1,F,R,62,2.0,NaN,28.0,0.5,1350,0.758,1.300,NaN
76,OAS1_0081_MR1,F,R,18,NaN,NaN,NaN,NaN,1309,0.857,1.341,NaN
326,OAS1_0361_MR1,M,R,20,NaN,NaN,NaN,NaN,1485,0.842,1.182,NaN
203,OAS1_0223_MR1,M,R,84,2.0,NaN,20.0,1.0,1641,0.703,1.070,NaN
4,OAS1_0005_MR1,M,R,18,NaN,NaN,NaN,NaN,1737,0.848,1.010,NaN
353,OAS1_0389_MR1,M,R,55,NaN,NaN,NaN,NaN,1678,0.782,1.046,NaN
364,OAS1_0402_MR1,F,R,76,3.0,2.0,30.0,0.5,1350,0.763,1.300,NaN
340,OAS1_0376_MR1,M,R,31,NaN,NaN,NaN,NaN,1579,0.817,1.111,NaN
244,OAS1_0271_MR1,F,R,89,2.0,4.0,27.0,0.0,1329,0.740,1.320,NaN
249,OAS1_0277_MR1,M,R,22,NaN,NaN,NaN,NaN,1913,0.841,0.917,NaN


In [5]:
oasis1.isnull().sum()

ID         0
M/F        0
Hand       0
Age        0
Educ     201
SES      220
MMSE     201
CDR      201
eTIV       0
nWBV       0
ASF        0
Delay    416
dtype: int64

## Data Cleaning

**We want to define a function that:**

1. Drops rows that have no values for CDR (prediction)
2. Fills empty values for socioeconomic status with a value
3. Replaces Male with 1 and Female with 0
4. Converts CDR from 4 unique values to 2 unique values so that we now have a binary classification problem
5. Drops columns Delay and Handedness

In [6]:
def clean_data(df):
    """This function removes CDR rows containing NANs, 
    fills SES missing values with status 3,
    and drops columns ID, Delay and Hand."""
    
    # Imputing with the median value 
    df['SES'].fillna(value=3, inplace=True) 
    
    # DROPPING ROWS WITHOUT TARGET
    df.dropna(subset = ["CDR"], inplace = True)
    
    
    # TARGET
    df["CDR"] = df["CDR"].map({0:0, 
                               0.5:1, 
                               1:1, 
                               2:1})
    
    # DROPPING USELESS FEATURES
    df.drop(['Delay', 'Hand', 'ID'], inplace = True, axis=1)

    return df.reset_index(drop = True)

In [7]:
oasis1 = clean_data(oasis1)
oasis1

,M/F,Age,Educ,SES,MMSE,CDR,eTIV,nWBV,ASF
0,F,74,2.0,3.0,29.0,0,1344,0.743,1.306
1,F,55,4.0,1.0,29.0,0,1147,0.810,1.531
2,F,73,4.0,3.0,27.0,1,1454,0.708,1.207
3,M,74,5.0,2.0,30.0,0,1636,0.689,1.073
4,F,52,3.0,2.0,30.0,0,1321,0.827,1.329
...,...,...,...,...,...,...,...,...,...
230,F,70,1.0,4.0,29.0,1,1295,0.748,1.355
231,F,73,3.0,2.0,23.0,1,1536,0.730,1.142
232,F,61,2.0,4.0,28.0,0,1354,0.825,1.297
233,M,61,5.0,2.0,30.0,0,1637,0.780,1.072


In [9]:
oasis1.Age

0      74
1      55
2      73
3      74
4      52
       ..
230    70
231    73
232    61
233    61
234    62
Name: Age, Length: 235, dtype: int64

In [ ]:
stop

### Testing my code

In [ ]:
# Check shape again
oasis1.shape

In [ ]:
# Education is not normally distrbuted
sns.histplot(oasis1.Educ,kde = True, color = 'c');

In [ ]:
sns.histplot(oasis1.SES, kde = True, color = 'c');

In [ ]:
# Check whether NANs are still here
oasis1.CDR.unique()

In [ ]:
# Check SES column
oasis1.SES.unique()

In [ ]:
# Check M/F column
oasis1['M/F']

## Defining variables

In [ ]:
# Defining variables
X = oasis1.drop(columns = ['CDR'])
y = oasis1['CDR']

In [ ]:
X

In [ ]:
# Holdout
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 42)

In [ ]:
X_train

In [ ]:
oasis1.isnull().sum()

## Preprocessing pipeline

📜 Note: columns **nWBV and ASF** already normalized
 
 According to visualizations in file 'Demographic_visual.ipynb', we decided to scale as follows:
 
**Columns to scale:**

Standard Scale: age

Robust Scale: eTIV, MMSE

MinMax Scale: Educ

In [ ]:
standard_features = ['Age']
robust_features = ['eTIV', 'MMSE']
minmax_features = ['Educ', 'SES']
nothing_to_scale = ['nWBV','ASF']
binary_categorical_features = ['M/F']

numerical_features = standard_features + robust_features + minmax_features + nothing_to_scale

In [ ]:
class CustomColumnTransformer(ColumnTransformer):
    def transform(self, *args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())
    def fit_transform(self, *args, **kwargs):
        return pd.DataFrame(super().fit_transform(*args, **kwargs), columns=self.get_feature_names_out())

In [ ]:
# Transformer for numerical features

def numerical_pipeline():
    """This function transforms all numerical features according to their respective scalers
    """
    
    # Simple Imputer
    # simple_imputer_frequent = SimpleImputer(strategy = "most_frequent")
    
    # Scalers
    standard_features = ['Age']
    robust_features = ['eTIV', 'MMSE']
    minmax_features = ['Educ', 'SES']
    nothing_to_scale = ['nWBV','ASF']
    
    features_in_this_order = standard_features + robust_features + minmax_features + nothing_to_scale
    
    numerical_transformer = CustomColumnTransformer([
        ('standard_scaler', StandardScaler(), standard_features),
        ('robust_scaler', RobustScaler(), robust_features),
        ('minmax_scaler', MinMaxScaler(), minmax_features),
         ], remainder = 'passthrough')
    
    
    # Pipeline
    numerical_pipeline = Pipeline([
        #("simple_imputer_most_frequent", simple_imputer_frequent),
        #("keeping_column_names", ColumnNameExtractorAfterImputer(features_in_this_order)),
        ("numerical_transformer", numerical_transformer)
    ])
    
    return numerical_pipeline

numerical_pipeline = numerical_pipeline()
numerical_pipeline

In [ ]:
# Transformer for categorical features

def binary_categorical_transformer():
    """This function encodes all categorical features according to their respective encoder.
    """
    categorical_transformer = CustomColumnTransformer([
       ("ohe_binary", 
         OneHotEncoder(sparse = False,
                       drop = "if_binary", 
                       handle_unknown = "error"), 
         binary_categorical_features)
    ])
    return categorical_transformer

binary_category_transformer = binary_categorical_transformer()
binary_category_transformer

## Parellel Transformation: Numerical + Categorical

In [ ]:
preprocessor = CustomColumnTransformer([
            ("num_transformer", numerical_pipeline, numerical_features ),
            ("cat_transformer", binary_category_transformer, binary_categorical_features)
            ])

preprocessor

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_train

In [ ]:
X_test = preprocessor.transform(X_test)
X_test

In [ ]:
# Length of y_test = 59
np.unique(y_test, return_counts = True)

# Length of X_test = 59
len(X_test)

In [ ]:
# Making a copy of X_scaled
X_train_trial = X_train.copy()
X_train_trial

# 🎰SVC

## SVC Linear

### Just a trial

In [ ]:
# Instantiating linear support vector classifier
model_svc = SVC(kernel = 'linear')

In [ ]:
# Fitting LinearSVC
model_svc.fit(X_train, y_train)

In [ ]:
# Prediction on X_test
y_pred_SVC = model_svc.predict(X_test)

In [ ]:
cm_svc = confusion_matrix(y_test, y_pred_SVC)
cm_svc

In [ ]:
baseline_recall_svc = recall_score(y_test, y_pred_SVC)
baseline_recall_svc

###   🤖  Grid Searching

In [ ]:
# Creating a parameter grid: map the parameter names to the values that should be searched in a dictionary
param_grid = {'C':np.arange(700,750,1),
              'gamma':[1,0.1,0.001,0.0001]}

# GridSearching
gridsearch_linear = GridSearchCV(model_svc, 
                          param_grid, n_jobs = -1, 
                          scoring = 'recall', cv = 5)

# Fitting gridsearch on X and y
gridsearch_linear.fit(X_train, y_train)

In [ ]:
# Best estimator obtained from grid searching
best_linear_svc = gridsearch_linear.best_estimator_
best_linear_svc

### Best SVC linear

In [ ]:
# Fitting the best estimator onto the train set
best_linear_svc.fit(X_train, y_train)

In [ ]:
# Predicting X_test
best_svc_pred = best_linear_svc.predict(X_test)

In [ ]:
# Confusion matrix
cm_svc = confusion_matrix(y_test, best_svc_pred)
cm_svc

disp = ConfusionMatrixDisplay(confusion_matrix=cm_svc,
                             display_labels=["Nondemented","Demented"])
disp.plot(cmap = 'viridis')
plt.grid(visible = None)
plt.show()

In [ ]:
# Scores
SVC_linear_score = recall_score(y_test, best_svc_pred), precision_score(y_test, best_svc_pred), accuracy_score(y_test, best_svc_pred)

## SVC Poly

###  🤖 Grid Searching

In [ ]:
# Creating a parameter grid: map the parameter names to the values that should be searched in a dictionary
param_grid = {'C':np.arange(1,50,1),
              'gamma':[1,0.1,0.001,0.0001],
             'degree': np.arange(1,10,1)}

# GridSearching
gridsearch_poly = GridSearchCV(SVC(kernel = 'poly'), 
                          param_grid, n_jobs = -1, 
                          scoring = 'recall', cv = 5)

# Fitting gridsearch on X and y
gridsearch_poly.fit(X_train, y_train)

In [ ]:
# Best estimator obtained from grid searching
best_poly_svc = gridsearch_poly.best_estimator_
best_poly_svc

### Best SVC Poly

In [ ]:
# Predicting X_test
best_poly_pred = best_poly_svc.predict(X_test)

In [ ]:
# Confusion matrix
cm_poly_svc = confusion_matrix(y_test, best_poly_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm_poly_svc,
                             display_labels=['Demented','Nondemented'])
disp.plot(cmap = 'viridis')
plt.grid(visible = None)
plt.show()

In [ ]:
# Scores
SVC_poly_score = recall_score(y_test, best_poly_pred), precision_score(y_test, best_poly_pred), accuracy_score(y_test, best_poly_pred)

## SVC Rbf

### Grid Searching

In [ ]:
# Creating a parameter grid: map the parameter names to the values that should be searched in a dictionary
param_grid = {'C':np.arange(1,200,1),
              'gamma':[1,0.1,0.5, 0.8, 0.001,0.0001]}

# GridSearching
gridsearch_rbf = GridSearchCV(SVC(kernel = 'rbf'), 
                          param_grid, n_jobs = -1, 
                          scoring = 'recall', cv = 5)

# Fitting gridsearch on X and y
gridsearch_rbf.fit(X_train, y_train)

In [ ]:
# Best estimator obtained from grid searching
best_rbf_svc = gridsearch_rbf.best_estimator_
best_rbf_svc

### Best SVC Rbf

In [ ]:
# Predicting X_test
best_rbf_pred = best_rbf_svc.predict(X_test)

In [ ]:
# Confusion matrix
cm_rbf_svc = confusion_matrix(y_test, best_rbf_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm_rbf_svc,
                             display_labels=['Demented','Nondemented'])
disp.plot(cmap = 'viridis')
plt.grid(visible = None)
plt.show()

In [ ]:
# Scores
SVC_rbf_score = recall_score(y_test, best_rbf_pred), precision_score(y_test, best_rbf_pred), accuracy_score(y_test, best_rbf_pred)

#  🤹 K Neighbors Classifier

###  💪🏻 Elbow Method

In [ ]:
# Instantiating KNeighborsClassifier
neigh = KNeighborsClassifier()

In [ ]:
# Elbow method
error_rate = []
for i in range(1,40):
 
 knn = KNeighborsClassifier(n_neighbors=i)
 knn.fit(X_train,y_train)
 pred_i = knn.predict(X_test)
 error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color= 'steelblue', linestyle= 'dashed', marker= 'o',
 markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('Number of K Neighbors', color = 'steelblue')
plt.ylabel('Error Rate', color = 'steelblue');

### Best Neigh model

In [ ]:
best_neigh = KNeighborsClassifier(n_neighbors = 5)
best_neigh

In [ ]:
# Fitting KNeighborsClassifier
best_neigh.fit(X_train, y_train)

In [ ]:
best_neigh_pred = best_neigh.predict(X_test)

In [ ]:
# Confusion matrix
cm_neigh = confusion_matrix(y_test, best_neigh_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm_neigh,
                             display_labels=['Demented','Nondemented'])
disp.plot(cmap = 'viridis')
plt.grid(visible = None)
plt.show()

In [ ]:
# Scores
neigh_score = recall_score(y_test, best_neigh_pred), precision_score(y_test, best_neigh_pred), accuracy_score(y_test, best_neigh_pred)

# Ensemble

## 🌳Decision Tree

### Trial (just to practice)

In [ ]:
np.unique(y_train, return_counts = True)

In [ ]:
# Instantiating Decision Tree
tree_clf = DecisionTreeClassifier(max_depth = 5)

In [ ]:
# Fitting decision tree
tree_clf.fit(X_train_trial,y_train)

In [ ]:
# Predicting X_test
prediction = tree_clf.predict(X_test)
prediction

In [ ]:
# Accuracy
recall_score(y_test, prediction)

In [ ]:
# Export model graph
export_graphviz(tree_clf, out_file="oasis1_tree.dot", 
                feature_names=X_train_trial.columns,
                class_names=['nondemented', 'demented'], 
                rounded=True,
                filled=True,
                rotate = True,
                fontname = 'futura')

# Import model graph
with open("oasis1_tree.dot") as f:
    dot_graph = f.read()
    display(graphviz.Source(dot_graph))

### 🤖 Grid Searching 

In [ ]:
# Parameters to grid search
parameters = {'criterion': ['gini', 'entropy'],
              'max_depth': range(1,10),
              'min_samples_split': range(2,10),
              'min_samples_leaf': range(2,10)
             #'max_features': range(0,len(X_train.columns),
              }

In [ ]:
# GridSearching
gridsearch_tree = GridSearchCV(DecisionTreeClassifier(), 
                          parameters, n_jobs = -1, 
                          scoring = 'recall', cv = 5)

# Fitting gridsearch on X and y
gridsearch_tree.fit(X_train, y_train)

In [ ]:
# Best score according to gridsearch
gridsearch_tree.best_score_

In [ ]:
# Best parameters
gridsearch_tree.best_params_

### 🌲 Best Tree

In [ ]:
# Instantiating best tree
best_tree = gridsearch_tree.best_estimator_

# Fitting best tree
best_tree.fit(X_train, y_train)

In [ ]:
# Predictions
y_tree_pred = best_tree.predict(X_test)
y_tree_pred

In [ ]:
# Scores
tree_score = recall_score(y_test, y_tree_pred), precision_score(y_test, y_tree_pred), accuracy_score(y_test, y_tree_pred)

In [ ]:
# Predictions on the test set
y_tree_pred = best_tree.predict(X_test)
y_tree_pred

In [ ]:
# Confusion Matrix
tree_cm = confusion_matrix(y_test, y_tree_pred)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=tree_cm,
                             display_labels=["nondemented","demented"])
disp.plot(cmap = 'plasma')

plt.grid(visible = False)
plt.show()

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()
# We want to capture all 29 people with dementia

In [ ]:
# Export model graph
export_graphviz(best_tree, out_file="best_tree.dot", 
                feature_names=X_train_trial.columns,
                class_names=['nondemented', 'demented'], 
                rounded=True,
                filled=True,
                rotate = True,
                fontname = 'futura')

# Import model graph
with open("best_tree.dot") as f:
    dot_graph = f.read()
    display(graphviz.Source(dot_graph))

<big> Conclusion: </big>

y_train = 76 demented, 100 nondemented

y_pred = 61 demented, 115 nondemented

## Random Forest

### Grid Searching

In [ ]:
# CREATING  RANDOM GRID
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 20, 
                               cv = 5, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)
rf_random.fit(X_train, y_train)

### Best Forest

In [ ]:
# Defining best estimator for random forest
best_rf = rf_random.best_estimator_

# Fitting best random forest
best_rf.fit(X_train, y_train)

In [ ]:
# Predictions on the test set
y_forest_pred = best_rf.predict(X_test)
y_forest_pred

In [ ]:
# Scores
forest_score = recall_score(y_test, y_forest_pred), precision_score(y_test, y_forest_pred), accuracy_score(y_test, y_forest_pred)

In [ ]:
# Confusion Matrix
rf_cm = confusion_matrix(y_test, y_forest_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=rf_cm,
                             display_labels=["nondemented","demented"])
disp.plot(cmap = 'plasma')

plt.grid(visible = False)
plt.show()

# AdaBoost Classifier

In [ ]:
adaboost = AdaBoostClassifier()

In [ ]:
hyperparameter_space = {'n_estimators':list(range(10, 100, 10)), 
                        'learning_rate':[0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}

gridsearch_ada = GridSearchCV(AdaBoostClassifier(base_estimator = adaboost,
                                     algorithm='SAMME.R',
                                     random_state=1),
                  param_grid=hyperparameter_space, 
                  scoring="recall", n_jobs=-1, cv=5)

gridsearch_ada.fit(X_train, y_train)
print("Optimal hyperparameter combination:", gridsearch_ada.best_params_)

In [ ]:
best_ada = AdaBoostClassifier(base_estimator = adaboost, n_estimators = 10, learning_rate = 0.6)

In [ ]:
best_ada.fit(X_train, y_train)

In [ ]:
# Predictions on the test set
y_ada_pred = best_ada.predict(X_test)
y_ada_pred

In [ ]:
# Scores
ada_score = recall_score(y_test, y_ada_pred), precision_score(y_test, y_ada_pred), accuracy_score(y_test, y_ada_pred)
ada_score

# XGBoost

- Sequential method
- Reduce bias
- Best weak learners given more weight

In [ ]:
# # Instantiating classifier
# xgb_cl = xgb.XGBClassifier(objective="binary:logistic", eval_metric = 'error')

# # Fit
# xgb_cl.fit(X_train, y_train)

# # Predict
# preds = xgb_cl.predict(X_test)

# # Score
# recall_score(y_test, preds)

### Grid Searching

In [ ]:
# param_grid = {
#     "max_depth": [3, 4, 5, 7],
#     "learning_rate": [0.1, 0.01, 0.05],
#     "gamma": [0, 0.25, 1],
#     "reg_lambda": [0, 1, 10],
#     "scale_pos_weight": [1, 3, 5],
#     "subsample": [0.8],
#     "colsample_bytree": [0.5],
# }

In [ ]:
# grid_cv = GridSearchCV(xgb_cl, 
#                        param_grid, 
#                        n_jobs=-1, 
#                        cv=3, 
#                        scoring="recall")

In [ ]:
# best_xgb = grid_cv.best_estimator_

In [ ]:
# best_xgb.fit(X_train, y_train)

In [ ]:
# # Predict
# best_xgb_pred = best_xgb.predict(X_test)

# # Score
# recall_score(y_test, best_xgb_pred)

# Exporting Data

In [ ]:
X_scaled = preprocessor.fit_transform(X).reset_index(drop=True)
X_scaled

In [ ]:
frames = [X_scaled, y]
preprocessed_oasis1 = pd.concat(frames, axis =1)

In [ ]:
#preprocessed_oasis1.to_csv(r'~/code/mkvph0ch/memobrain/notebooks/preprocessed_oasis1.csv')

# Recall Scores

In [ ]:
print("RECALL, PRECISION, ACCURACY")
(SVC_linear_score, SVC_poly_score, SVC_rbf_score, neigh_score, tree_score, forest_score, ada_score)

<big> <b> Model names </b> </big>

<big> <font color = 'steelblue'> SVC </color> </big>

Linear: best_linear_svc - 78.2


Polynomial: best_poly_svc - 69.6 
    

Rbf: best_rbf_svc - 82.6  


<big> <font color = 'steelblue'> K Neighbors Classifier </color> <big>

best_neigh - 65.0

<big> <font color = 'steelblue'> Decision Tree </color> </big>
    
best_tree - 78.3


# Evaluating OASIS2

## Importing Data

In [ ]:
oasis2_cs_path = '../raw_data/OASIS2/preprocessed_oasis2.csv'
oasis2 = pd.read_csv(oasis2_cs_path)

In [ ]:
oasis2

In [ ]:
X2 = oasis2.drop(columns = 'CDR')
y2 = oasis2['CDR']

In [ ]:
X_train2, X_test2, ytrain2, y_test2 = train_test_split(X2, y2, test_size=0.30, random_state=42)

## Evaluating

In [ ]:
best_pred = best_rf.predict(X2)
best_pred

In [ ]:
recall_score(y2, best_pred)